# Annotations of the workflow

The idea is to get more content about the project/workflow.

* Conda environment for this Notebook: [Greference_tools](../../environments/Greference_tools.yml)

In [27]:
## Import libreries

# Base packages
import subprocess
import os
import re
import random
# Data manipulation
import pandas as pd
import numpy as np

In [ ]:
random.seed(97)

# Metadata/report.csv

Lets see our Project and all the fields.

For this project we will be interested in download the Bam files in the ***8 field***: **submitted_ftp**

In [2]:
report = pd.read_csv("../../../metadata/report.tsv", sep="\t")
report

,study_accession,sample_accession,experiment_accession,run_accession,tax_id,scientific_name,fastq_ftp,submitted_ftp,sra_ftp
0,PRJEB7926,SAMEA3168442,ERX641122,ERR696683,9606,Homo sapiens,ftp.sra.ebi.ac.uk/vol1/fastq/ERR696/ERR696683/...,ftp.sra.ebi.ac.uk/vol1/run/ERR696/ERR696683/C2...,NaN
1,PRJEB7926,SAMEA3168443,ERX697048,ERR753368,9606,Homo sapiens,ftp.sra.ebi.ac.uk/vol1/fastq/ERR753/ERR753368/...,ftp.sra.ebi.ac.uk/vol1/run/ERR753/ERR753368/C2...,NaN
2,PRJEB7926,SAMEA3168444,ERX697049,ERR753369,9606,Homo sapiens,ftp.sra.ebi.ac.uk/vol1/fastq/ERR753/ERR753369/...,ftp.sra.ebi.ac.uk/vol1/run/ERR753/ERR753369/C2...,NaN
3,PRJEB7926,SAMEA3168445,ERX697050,ERR753370,9606,Homo sapiens,ftp.sra.ebi.ac.uk/vol1/fastq/ERR753/ERR753370/...,ftp.sra.ebi.ac.uk/vol1/run/ERR753/ERR753370/C3...,NaN
4,PRJEB7926,SAMEA3168446,ERX697051,ERR753371,9606,Homo sapiens,ftp.sra.ebi.ac.uk/vol1/fastq/ERR753/ERR753371/...,ftp.sra.ebi.ac.uk/vol1/run/ERR753/ERR753371/C3...,NaN
5,PRJEB7926,SAMEA3168447,ERX697052,ERR753372,9606,Homo sapiens,ftp.sra.ebi.ac.uk/vol1/fastq/ERR753/ERR753372/...,ftp.sra.ebi.ac.uk/vol1/run/ERR753/ERR753372/C3...,NaN
6,PRJEB7926,SAMEA3168448,ERX697053,ERR753373,9606,Homo sapiens,ftp.sra.ebi.ac.uk/vol1/fastq/ERR753/ERR753373/...,ftp.sra.ebi.ac.uk/vol1/run/ERR753/ERR753373/NG...,NaN
7,PRJEB7926,SAMEA3168449,ERX697054,ERR753374,9606,Homo sapiens,ftp.sra.ebi.ac.uk/vol1/fastq/ERR753/ERR753374/...,ftp.sra.ebi.ac.uk/vol1/run/ERR753/ERR753374/NG...,NaN
8,PRJEB7926,SAMEA3168450,ERX697055,ERR753375,9606,Homo sapiens,ftp.sra.ebi.ac.uk/vol1/fastq/ERR753/ERR753375/...,ftp.sra.ebi.ac.uk/vol1/run/ERR753/ERR753375/C-...,NaN
9,PRJEB7926,SAMEA3168451,ERX697056,ERR753376,9606,Homo sapiens,ftp.sra.ebi.ac.uk/vol1/fastq/ERR753/ERR753376/...,ftp.sra.ebi.ac.uk/vol1/run/ERR753/ERR753376/C2...,NaN


## Downloading the table 3, for supporting information found in the article.

After that we will use the function in pandas ```pd.read_clipboard()``` to download the data.

https://onlinelibrary.wiley.com/action/downloadSupplement?doi=10.1002%2Fgcc.22314&file=gcc22314-sup-0005-supptable5.docx

In [26]:
table3=pd.read_clipboard()
table3.head()

,Sample,Raw Reads,Sequencing depth (X),Reads after QC,Reads after QC# %,Mapped reads,Reads after QC %,% On-target (30X),Mean coverage (X),Genes covered (%),SNPs,Indels
0,IV-5^,29 191 671,84.95,22 978 888,"79,00 %",45 770 776,99.59%,"49,00 %",17.79x(1x-856x),98.48%,114 651,6 935
1,IV-2^,40 672 795,122.02,39 920 701,"88,00 %",71 507 872,99.54%,"79,00 %",26.76x(1x-860x),99.70%,226 812,10 226
2,IV-6^,43 486 476,130.46,38 353 481,"88,00 %",76 404 013,99.61%,"81,00 %",29.66x(1x-817x),99.71%,207 627,10 263
3,IV-4^,42 838 012,128.51,37 729 408,"88,00 %",75 125 893,99.56%,"81,00 %",28.76x(1x-877x),99.91%,229 099,10 325
4,IV-7^,377 188 93,113.16,33 250 705,"88,00 %",66 224 206,99.58%,"77,00 %",25.82x(1x-847x),99.70%,203 338,9 474


In [27]:
## We wolud do some tranformations to the first field ("Sample")
table3["Sample"] = table3["Sample"].str.replace(
    "-", "_"
    ).str.replace(
        "^", "_method1"
        ).str.replace(
            "*", "_method2"
            )
table3.head()

/tmp/ipykernel_34176/3769299083.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  ).str.replace(
/tmp/ipykernel_34176/3769299083.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  ).str.replace(


,Sample,Raw Reads,Sequencing depth (X),Reads after QC,Reads after QC# %,Mapped reads,Reads after QC %,% On-target (30X),Mean coverage (X),Genes covered (%),SNPs,Indels
0,IV_5_method1,29 191 671,84.95,22 978 888,"79,00 %",45 770 776,99.59%,"49,00 %",17.79x(1x-856x),98.48%,114 651,6 935
1,IV_2_method1,40 672 795,122.02,39 920 701,"88,00 %",71 507 872,99.54%,"79,00 %",26.76x(1x-860x),99.70%,226 812,10 226
2,IV_6_method1,43 486 476,130.46,38 353 481,"88,00 %",76 404 013,99.61%,"81,00 %",29.66x(1x-817x),99.71%,207 627,10 263
3,IV_4_method1,42 838 012,128.51,37 729 408,"88,00 %",75 125 893,99.56%,"81,00 %",28.76x(1x-877x),99.91%,229 099,10 325
4,IV_7_method1,377 188 93,113.16,33 250 705,"88,00 %",66 224 206,99.58%,"77,00 %",25.82x(1x-847x),99.70%,203 338,9 474


In [30]:
table3.to_csv("../../../metadata/table3.csv", index=None)

In [3]:
table3_csv = pd.read_csv("../../../metadata/table3.csv")
table3_csv

,Sample,Raw Reads,Sequencing depth (X),Reads after QC,Reads after QC# %,Mapped reads,Reads after QC %,% On-target (30X),Mean coverage (X),Genes covered (%),SNPs,Indels
0,IV_5_method1,29 191 671,84.95,22 978 888,"79,00 %",45 770 776,99.59%,"49,00 %",17.79x(1x-856x),98.48%,114 651,6 935
1,IV_2_method1,40 672 795,122.02,39 920 701,"88,00 %",71 507 872,99.54%,"79,00 %",26.76x(1x-860x),99.70%,226 812,10 226
2,IV_6_method1,43 486 476,130.46,38 353 481,"88,00 %",76 404 013,99.61%,"81,00 %",29.66x(1x-817x),99.71%,207 627,10 263
3,IV_4_method1,42 838 012,128.51,37 729 408,"88,00 %",75 125 893,99.56%,"81,00 %",28.76x(1x-877x),99.91%,229 099,10 325
4,IV_7_method1,377 188 93,113.16,33 250 705,"88,00 %",66 224 206,99.58%,"77,00 %",25.82x(1x-847x),99.70%,203 338,9 474
5,V_8_method1,55 720 364,222.88,51 583 390,"93,00 %",102 574 419,99.43%,"82,00 %",49.28x(1x-997x),99.57%,195 153,12 768
6,V_2_method1,55 054 777,160.22,37 529 435,"68,00 %",74 739 463,99.57%,"64,00 %",27.24x(1x-998x),98.62%,137 021,8 655
7,V_1_method1,57 215 132,166.51,34 333 842,"60,00 %",68 423 638,99.64%,"38,00 %",14.35x(1x-885x),98.21%,99 034,5 679
8,IV_6_method2,43486476,"130,46",38 353 481,"88,00 %",76 404 013,"99,61 %","81,00 %",29.66x(1x-817x),"99,71 %",207 626,10 263
9,IV_2_method2,40672795,"122,2",35920701,"88,00 %",71 507 872,"99,54 %","79,00 %",26.76x(1x-860x),"99,70 %",226 812,10 226


### Amount of reads after removing low quality sequences. 
### Low quality sequences are considered when

1. bases with Q20 account for >85% of the total sequence length 
2. reads with N-rate >10%

### Methods for seguencing

* **method1** run with Agilent Sureselect XT Human All Exon 50Mb
* **method2** run with Agilent SureSelect_XT_Human_All_Exon_V5

## Mean depth of the project

In [4]:
table3_csv["Sequencing depth (X)"].str.replace(",",".").astype(float).mean()

135.00333333333336

## **Plot of the rules from the snakemake pipeline**

In [5]:
subprocess.run(["python", "../scripts_analysis/workflow_viz.py"])

CompletedProcess(args=['python', '../scripts_analysis/workflow_viz.py'], returncode=0)

<p align="center">
 
  <img src="../results_analysis/plots/snakemake_rules_linear.png" alt="600" width="500" />

</p>

Plot to show the structure to follow of the pipline

* Color *green*: you have to execute **multiple** times this rule (as much chromosomes/genes you want to filter).

* Color *orange*: you have to execute only **1** time, the first run of the workflow.

* Color *red*: you have to execute only **1** time, the last run, after having all genes filter tables.

## **Calculating the AVERAGE sequence length for each sample**  

In [6]:
## heavy computing +25 min -> you need all fastq.gz files in data/raw/

for chr in ["chr3", "chr5", "chr7", "chr12", "chr17"]:
    subprocess.run(["bash", "../scripts_analysis/01calculating_length.sh", chr])

subprocess.run(["bash", "../scripts_analysis/02joining_files.sh"])


CompletedProcess(args=['bash', '../scripts_analysis/02joining_files.sh'], returncode=0)

In [7]:
list_files = os.listdir("../../../results/annotations/")

list_files_path = []

for file in range(len(list_files)):
    path = "../../../results/annotations/" + list_files[file]
    list_files_path.append(path)

In [8]:
list_mean_len = []
list_sd_len = []

for file_path in list_files_path:

    with open(file_path) as file:
        len_list = file.readlines()
    
    mean_seq_len = np.array([s.strip() for s in len_list]).astype(int).mean().round(2)
    sd_seq_len = np.array([s.strip() for s in len_list]).astype(int).std().round(2)
    
    print("mean value of ", file_path.replace(
        "../../../results/annotations/", ""
        ).replace(
            "_len.txt", ""
            ), " -> ", mean_seq_len, "+/-", sd_seq_len)

    list_mean_len.append(mean_seq_len)
    list_sd_len.append(sd_seq_len)


mean value of  ERR753374  ->  94.61 +/- 2.29
mean value of  ERR753373  ->  94.72 +/- 2.18
mean value of  ERR753375  ->  75.51 +/- 1.26
mean value of  ERR753372  ->  75.89 +/- 0.73
mean value of  ERR753370  ->  100.64 +/- 1.27
mean value of  ERR696683  ->  75.48 +/- 1.31
mean value of  ERR753376  ->  75.48 +/- 1.31
mean value of  ERR753378  ->  75.89 +/- 0.73
mean value of  ERR753371  ->  75.24 +/- 1.53
mean value of  ERR753369  ->  75.89 +/- 0.73
mean value of  ERR753377  ->  75.5 +/- 1.29
mean value of  ERR753368  ->  75.89 +/- 0.73


In [9]:
df_len_seq = pd.DataFrame({
    "sample": pd.Series(list_files_path),
    "mean_len": pd.Series(list_mean_len),
    "sd_len": pd.Series(list_sd_len)
})

df_len_seq["sample"] = df_len_seq["sample"].str.replace("../../results/annotations/", "")
df_len_seq["sample"] = df_len_seq["sample"].str.replace("_len.txt", "")

df_len_seq.to_csv("../results_analysis/tables/len_mean.csv", index=None)

/tmp/ipykernel_19023/178359344.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df_len_seq["sample"] = df_len_seq["sample"].str.replace("../../results/annotations/", "")
/tmp/ipykernel_19023/178359344.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df_len_seq["sample"] = df_len_seq["sample"].str.replace("_len.txt", "")


In [10]:
subprocess.run(["Rscript", "../scripts_analysis/03plot_mean_len.R"])

Rows: 12 Columns: 3
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): sample
dbl (2): mean_len, sd_len

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


CompletedProcess(args=['Rscript', '../scripts_analysis/03plot_mean_len.R'], returncode=0)

<p align="center">
 
  <img src="../results_analysis/plots/mean_len.png" alt="800" width="900" />

</p>

## **Number of Seqs before and after fastp**

* In order to run the next code, you have to have all fastq files (**all chromosomes, reads**) in *raw* and *processed*

In [11]:
subprocess.run(["bash", "../scripts_analysis/04fastp_analysis.sh"])

CompletedProcess(args=['bash', '../scripts_analysis/04fastp_analysis.sh'], returncode=0)

In [12]:
df_fastp = pd.read_csv("../results_analysis/tables/fastp_analysis.tsv", sep="\t")
df_fastp

,nseqs_before_fastp_1,nseqs_after_fastp_1,nseqs_before_fastp_2,nseqs_after_fastp_2,sample
0,12782098,7326944,13710428,7805056,ERR696683
1,13599176,11157220,14455979,11949382,ERR753368
2,14418846,11807880,15425862,12735619,ERR753369
3,19137007,19062314,19599533,19503957,ERR753370
4,14433503,7201499,12561966,5931469,ERR753371
5,14631641,11966864,15577190,12843549,ERR753372
6,13156883,12736478,13659821,13173367,ERR753373
7,8358030,8054428,8337180,8001535,ERR753374
8,7548397,4585721,7793064,4706718,ERR753375
9,10409144,5947899,11104110,6292675,ERR753376


## Mapped reads, with the reference Genome

* ### flagstats: in order to do this analysis, you need all the flagstats information in metadata/logs/flagstats.

In [13]:
subprocess.run(["bash", "../scripts_analysis/05flagstats.sh"])

CompletedProcess(args=['bash', '../scripts_analysis/05flagstats.sh'], returncode=0)

In [14]:
flagstats = pd.read_csv("../results_analysis/tables/flagstats.tsv", sep="\t")
flagstats["mapped_reads_per"] = np.array(flagstats.mapped_reads/flagstats.num_seqs_afterQC*100).round(2)
flagstats["properly_paired_per"] = np.array(flagstats.properly_mapped/flagstats.num_seqs_afterQC*100).round(2) 
flagstats["singletons_per"] = np.array(flagstats.singletons/flagstats.num_seqs_afterQC*100).round(2)  

In [15]:
flagstats.to_csv("../results_analysis/tables/flagstats.tsv", sep="\t")

In [16]:
flagstats

,num_seqs_afterQC,mapped_reads,properly_mapped,singletons,sample,mapped_reads_per,properly_paired_per,singletons_per
0,11412296,11409623,11231544,2627,ERR696683,99.98,98.42,0.02
1,16198266,16196036,15993996,2168,ERR753368,99.99,98.74,0.01
2,17277399,17275308,17129464,2035,ERR753369,99.99,99.14,0.01
3,26218218,26212327,25691172,5811,ERR753370,99.98,97.99,0.02
4,8699690,8698945,8641504,725,ERR753371,99.99,99.33,0.01
5,17248693,17246266,17053104,2369,ERR753372,99.99,98.87,0.01
6,15265510,15264271,14760714,1183,ERR753373,99.99,96.69,0.01
7,8999288,8998390,8664044,870,ERR753374,99.99,96.27,0.01
8,7405222,7404251,7339372,947,ERR753375,99.99,99.11,0.01
9,9079860,9078482,8990836,1342,ERR753376,99.98,99.02,0.01


### Plotting the previous table

In [17]:
subprocess.run(["Rscript", "../scripts_analysis/06plot_flagstats.R"])

New names:
• `` -> `...1`
Rows: 12 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (1): sample
dbl (8): ...1, num_seqs_afterQC, mapped_reads, properly_mapped, singletons, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


CompletedProcess(args=['Rscript', '../scripts_analysis/06plot_flagstats.R'], returncode=0)

<p align="center">
 
  <img src="../results_analysis/plots/flagstats.png" alt="800" width="900" />

</p>

## VCF stats

* For this analysis you'll need all the files created in the dir metadata/logs/vcfstats  

In [18]:
subprocess.run(["bash", "../scripts_analysis/07vcfstats.sh"])

CompletedProcess(args=['bash', '../scripts_analysis/07vcfstats.sh'], returncode=0)

In [19]:
## Process additional vcf stats 
vcf_stats = pd.read_csv("../results_analysis/tables/additional_vcf_stats.tsv", sep="\t", index_col=False)
vcf_stats

,snp_transition_transversion,total_het_hom,snp_het_hom,mnp_het_hom,Insert_het_hom,del_het_hom,Indel_het_hom,insert_del,indel_snp_plus_mnp,sample
0,16969|7054,10718|7898,9729|7144,254|104,323|322,389|302,23|26,645|691,1385|17231,ERR696683
1,22790|9589,14288|10835,12891|9739,371|148,480|469,497|447,49|32,949|944,1974|23149,ERR753368
2,22316|9338,14172|10542,12726|9458,368|167,472|462,560|422,46|33,934|982,1995|22719,ERR753369
3,27389|11651,17994|12977,16011|11510,484|176,614|615,807|626,78|50,1229|1433,2790|28181,ERR753370
4,12226|4926,7329|5749,6641|5254,208|79,198|205,266|196,16|15,403|462,896|12182,ERR753371
5,22773|9638,14807|10579,13330|9536,391|141,475|439,559|427,52|36,914|986,1988|23398,ERR753372
6,25705|12262,12251|14997,10881|13538,369|210,438|598,522|603,41|48,1036|1125,2250|24998,ERR753373
7,14105|6359,7057|7935,6290|7083,198|116,260|338,292|368,17|30,598|660,1305|13687,ERR753374
8,14455|5812,9045|6578,8214|6023,231|66,267|227,309|240,24|22,494|549,1089|14534,ERR753375
9,15429|6239,9113|7376,8257|6705,224|94,265|261,342|287,25|29,526|629,1209|15280,ERR753376


In [20]:
new_cols = []
for col in vcf_stats.iloc[:,:9].columns:
    new_cols.append("r_" + col)

print(new_cols)

for name,ncol in zip(new_cols, range(len(vcf_stats.columns) -1)):
    vcf_stats[name] = vcf_stats.iloc[:,ncol].str.split("|").apply(lambda x: float(x[0]) / float(x[1])).round(2)

vcf_stats

ratios_vstats = vcf_stats.iloc[:,9:]

['r_snp_transition_transversion', 'r_total_het_hom', 'r_snp_het_hom', 'r_mnp_het_hom', 'r_Insert_het_hom', 'r_del_het_hom', 'r_Indel_het_hom', 'r_insert_del', 'r_indel_snp_plus_mnp']


In [21]:
for name in vcf_stats.iloc[:,:9]:
    vcf_stats[name] = "(" + vcf_stats[name].str.replace("|", "/") + ")"
    
for name,r_name in zip(vcf_stats.iloc[:,:9].columns, vcf_stats.iloc[:,10:].columns):
    vcf_stats[name] = vcf_stats[r_name].astype(str) + " " + vcf_stats[name]
    

/tmp/ipykernel_19023/1472517112.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  vcf_stats[name] = "(" + vcf_stats[name].str.replace("|", "/") + ")"


In [22]:
vcf_stats = vcf_stats.iloc[:,:10]

vcf_stats.to_csv("../results_analysis/tables/additional_vcf_stats.tsv")


In [23]:
vcf_stats

,snp_transition_transversion,total_het_hom,snp_het_hom,mnp_het_hom,Insert_het_hom,del_het_hom,Indel_het_hom,insert_del,indel_snp_plus_mnp,sample
0,2.41 (16969/7054),1.36 ( 10718/7898),1.36 ( 9729/7144),2.44 ( 254/104),1.0 ( 323/322),1.29 ( 389/302),0.88 ( 23/26),0.93 ( 645/691),0.08 ( 1385/17231),ERR696683
1,2.38 (22790/9589),1.32 ( 14288/10835),1.32 ( 12891/9739),2.51 ( 371/148),1.02 ( 480/469),1.11 ( 497/447),1.53 ( 49/32),1.01 ( 949/944),0.09 ( 1974/23149),ERR753368
2,2.39 (22316/9338),1.34 ( 14172/10542),1.35 ( 12726/9458),2.2 ( 368/167),1.02 ( 472/462),1.33 ( 560/422),1.39 ( 46/33),0.95 ( 934/982),0.09 ( 1995/22719),ERR753369
3,2.35 (27389/11651),1.39 ( 17994/12977),1.39 ( 16011/11510),2.75 ( 484/176),1.0 ( 614/615),1.29 ( 807/626),1.56 ( 78/50),0.86 ( 1229/1433),0.1 ( 2790/28181),ERR753370
4,2.48 (12226/4926),1.27 ( 7329/5749),1.26 ( 6641/5254),2.63 ( 208/79),0.97 ( 198/205),1.36 ( 266/196),1.07 ( 16/15),0.87 ( 403/462),0.07 ( 896/12182),ERR753371
5,2.36 (22773/9638),1.4 ( 14807/10579),1.4 ( 13330/9536),2.77 ( 391/141),1.08 ( 475/439),1.31 ( 559/427),1.44 ( 52/36),0.93 ( 914/986),0.08 ( 1988/23398),ERR753372
6,2.1 (25705/12262),0.82 ( 12251/14997),0.8 ( 10881/13538),1.76 ( 369/210),0.73 ( 438/598),0.87 ( 522/603),0.85 ( 41/48),0.92 ( 1036/1125),0.09 ( 2250/24998),ERR753373
7,2.22 (14105/6359),0.89 ( 7057/7935),0.89 ( 6290/7083),1.71 ( 198/116),0.77 ( 260/338),0.79 ( 292/368),0.57 ( 17/30),0.91 ( 598/660),0.1 ( 1305/13687),ERR753374
8,2.49 (14455/5812),1.38 ( 9045/6578),1.36 ( 8214/6023),3.5 ( 231/66),1.18 ( 267/227),1.29 ( 309/240),1.09 ( 24/22),0.9 ( 494/549),0.07 ( 1089/14534),ERR753375
9,2.47 (15429/6239),1.24 ( 9113/7376),1.23 ( 8257/6705),2.38 ( 224/94),1.02 ( 265/261),1.19 ( 342/287),0.86 ( 25/29),0.84 ( 526/629),0.08 ( 1209/15280),ERR753376


In [24]:
ratios_vstats.to_csv("../results_analysis/tables/ratios_vcfstats.tsv", sep="\t")
ratios_vstats.head()

,sample,r_snp_transition_transversion,r_total_het_hom,r_snp_het_hom,r_mnp_het_hom,r_Insert_het_hom,r_del_het_hom,r_Indel_het_hom,r_insert_del,r_indel_snp_plus_mnp
0,ERR696683,2.41,1.36,1.36,2.44,1.00,1.29,0.88,0.93,0.08
1,ERR753368,2.38,1.32,1.32,2.51,1.02,1.11,1.53,1.01,0.09
2,ERR753369,2.39,1.34,1.35,2.20,1.02,1.33,1.39,0.95,0.09
3,ERR753370,2.35,1.39,1.39,2.75,1.00,1.29,1.56,0.86,0.10
4,ERR753371,2.48,1.27,1.26,2.63,0.97,1.36,1.07,0.87,0.07


In [25]:
subprocess.run(["Rscript", "../scripts_analysis/08ploting_vcfstats.R"])

Rows: 12 Columns: 8
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (2): pass_fail_filter, sample
dbl (6): snps, mnps, insertions, deletions, indels, same_reference

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
New names:
• `` -> `...1`
Rows: 12 Columns: 11
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr  (1): sample
dbl (10): ...1, r_snp_transition_transversion, r_total_het_hom, r_snp_het_ho...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Warning message:
Using `size` aesthetic for lines was deprecated in ggplot2 3.4.0.
ℹ Please use `linewidth` instead. 


CompletedProcess(args=['Rscript', '../scripts_analysis/08ploting_vcfstats.R'], returncode=0)

<p align="center">
 
  <img src="../results_analysis/plots/vcf_basic_stats.png" alt="800" width="900" />

</p>

<p align="center">
 
  <img src="../results_analysis/plots/ratios_vcf.png" alt="800" width="900" />

</p>

<p align="center">
 
  <img src="../results_analysis/plots/ratios_het_hom_total_vcf.png" alt="800" width="900" />

</p>

<p align="center">
 
  <img src="../results_analysis/plots/ratios_het_hom_vcf.png" alt="800" width="900" />

</p>

In [26]:
## Statisticall analysis from the last plot
subprocess.run(["Rscript", "../scripts_analysis/09statistical_analysis.R"])

[1] "===> Total values"
[1] 60
[1] "===> Total values pre group"
# A tibble: 5 × 2
  het_hom              n
  <chr>            <int>
1 r_Indel_het_hom     12
2 r_Insert_het_hom    12
3 r_del_het_hom       12
4 r_mnp_het_hom       12
5 r_snp_het_hom       12
[1] "=------------------------------------="
[1] "===>           Normality          <==="
[1] "=------------------------------------="

	Shapiro-Wilk normality test

data:  vcf_stats$num
W = 0.8368, p-value = 1.27e-06

# A tibble: 5 × 4
  het_hom          variable statistic        p
  <chr>            <chr>        <dbl>    <dbl>
1 r_Indel_het_hom  num          0.929 0.365   
2 r_Insert_het_hom num          0.913 0.231   
3 r_del_het_hom    num          0.786 0.00658 
4 r_mnp_het_hom    num          0.946 0.584   
5 r_snp_het_hom    num          0.689 0.000667
[1] "=------------------------------------="
[1] "===>      Kruskal_Wallis/Dunnet   <==="
[1] "=------------------------------------="
# A tibble: 1 × 6
  .y.       n statistic

CompletedProcess(args=['Rscript', '../scripts_analysis/09statistical_analysis.R'], returncode=0)

<p align="center">
 
  <img src="../results_analysis/plots/statistics_vcf_inference.png" alt="800" width="900" />

</p>
